In [1]:
import nbformat
import re

In [2]:
in_filename = 'portfolio/fundamentals/011-sklearn-classification-solutions.ipynb'

In [3]:
nb = nbformat.read(in_filename, as_version=nbformat.NO_CONVERT)

In [4]:
cells = []
for cell in nb['cells']:
    if cell['cell_type'] != 'code':
        cells.append(cell)
        continue
    if cell.source.startswith("# remove"):
        continue
    cells.append(cell)
nb['cells'] = cells

In [5]:
source = cells[28].source
print(source)

**Fit a decision tree classifier (call it `dtree_clf`) to the training set**. Use the default hyperparameters.


In [6]:
def strip_source(source):
    result = []
    lines = source.split('\n')
    skip_next = False
    skip_until = None
    for i in range(len(lines)):
        if skip_next:
            skip_next = False
            continue
        line = lines[i]
        if skip_until is not None:
            if skip_until in line:
                # all done
                skip_until = None
            continue
        if i == len(lines) - 1:
            # last time unconditional
            result.append(line)
            continue
        if 'your code here' in line:
            result.append(line)
            skip_until = '-'*1000 # FIXME: weird sentinel
            continue
        is_commented = line.strip().startswith("# ")
        if is_commented:
            uncommented = re.sub(r'^(\s*)(# )(.+)$', r'\1\3', line)

            heredoc_match = re.match(r'(.+)\s+<<(.+)', uncommented)
            if heredoc_match is not None:
                result.append(heredoc_match.group(1))
                end_str = heredoc_match.group(2).strip()
                skip_until = end_str
                continue

            if '...' in line:
                # probable blank-filler
                match_re = '^' + re.escape(uncommented).replace(r'\.\.\.', '(.+)') + "$"
                if re.match(match_re, lines[i+1]):
                    skip_next = True
                    result.append(uncommented)
                    continue
                else:
                    print("WARNING: failed match", uncommented)
        result.append(line)
    return '\n'.join(result)
strip_source(source)

'**Fit a decision tree classifier (call it `dtree_clf`) to the training set**. Use the default hyperparameters.'

In [8]:
for cell in cells:
    if cell['cell_type'] == 'code':
        cell.source = strip_source(cell.source)
        cell.outputs.clear()

In [9]:
out_fname = in_filename.replace('-solutions', '')
if out_fname == in_filename:
    out_fname = in_filename + '.out'

In [10]:
out_fname

'portfolio/fundamentals/011-sklearn-classification.ipynb'

In [11]:
nbformat.write(nb, out_fname)